In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import scipy.stats
import anndata
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.axes._axes import _log as matplotlib_axes_logger
from scipy import sparse
matplotlib_axes_logger.setLevel('ERROR')

In [2]:
target_genes = pd.read_csv('/home/jovyan/scripts/renal_covid_19/sig_gene_plot/Macrophage_stim_markers_Human.csv')

In [3]:
target_genes.head()

,GC,HDL,IFNb,IFNg,IFNg_TNF,IL10,IL13,IL4,IL4_upLPS,LA,...,TNF_P3C,TNF_PGE2,TPP,TPP_IFNb,TPP_IFNb_IFNg,sLPS,sLPS_IC,sLPS_IFNg,upLPS,upLPS_IC
0,GLDN,LOC653352,C9orf38,GBP5,CSF1,HCFC1R1,ALOX15,ALOX15,ALOX15,MMP9,...,LRG1,GBP5,RGS16,C1QTNF1,PAMR1,CXCR7,MYBPC3,GBP5,PCNX,SRPX
1,SRPX,YIPF6,ACY3,LOC400759,CCL5,SPOCD1,FOXQ1,FOXQ1,IL17RB,UCK2,...,CXCR5,PCNX,NFS1,ZSWIM4,CXCR7,PTPRF,IL2RA,PCNX,ANG,TBL2
2,ZBTB16,MOGAT1,MGAT4B,CXCL9,C20orf123,P8,PPP1R14A,PPP1R14A,CCL26,FABP5,...,ALPL,GBP4,CCL5,PCNX,PTPRF,FAM108C1,CXCR7,GCH1,RNASE4,LRG1
3,TBC1D16,PRPF40A,LOC732424,GBP1,IL2RG,ADAMTSL4,DHRS11,MAOA,WNT5A,MMP7,...,CXCR7,RCN1,CYP27B1,FAM108C1,C1QTNF1,KIAA1199,PCNX,LOC399715,SPARC,MS4A6E
4,LOC647108,LOC642033,PITX2,GBP4,GBP4,CTSG,IL1R1,SOCS1,TGM2,ACADVL,...,NFS1,SLC5A3,GPR124,CXCR7,ZSWIM4,EMR3,MLLT6,MYBPC3,FTHL12,C1S


In [4]:
mdata = sc.read('/home/jovyan/scripts/renal_covid_19/steroid_pipeline/trajectory_with_Rik_model_output/wave2_steroid_2021_covid_GPLVM.h5ad') #inlcude all myeloid annoted
#adata = sc.read('/home/jovyan/mounting_point/renal_covid_19_gex_myeloid_soupX_QC_orig.ident_annotated_levle23.h5ad')

In [5]:
mdata

AnnData object with n_obs × n_vars = 17256 × 32913
    obs: 'annot4', 'centre', 'sample_id', 'sample_id_broad', 'sample_date', 'sample_date_yr', 'pool', 'pool_broad', 'haniffa_broad_predLabel', 'orig.ident', 'merged_souporcell_cluster', 'merged_souporcell_status', 'patient_id', 'case_control', 'WHO_severity', 'WHO_temp_severity', 'sex', 'calc_age', 'discharge_date', 'date_positive_swab', 'date_first_symptoms', 'admission_date', 'ethnicity', 'individual_id', 'pseudotime_GPLVM', 'pseudobatch_GPLVM', 'steroid_date', 'days_from_steroid', 'time_from_first_symptoms', 'time_from_positive_swab', 'time_from_infection'
    var: 'GEX'

In [6]:
adata_gex=sc.read('/home/jovyan/data/covid_19_renal/renal_covid_19_gex_myeloid_soupX_QC.h5ad')

In [7]:
adata_gex

AnnData object with n_obs × n_vars = 220907 × 33559

In [8]:
adata = adata_gex[adata_gex.obs.index.isin(mdata.obs.index)]

In [9]:
adata

View of AnnData object with n_obs × n_vars = 17256 × 33559

In [10]:
adata.obs['annot4'] = mdata.obs['annot4']
adata.obs['centre'] = mdata.obs['centre']
adata.obs['case_control'] = mdata.obs['case_control']
adata.obs['WHO_severity'] = mdata.obs['WHO_severity']
adata.obs['days_from_steroid'] = mdata.obs['days_from_steroid']
adata.obs['sample_id'] = mdata.obs['sample_id']
adata.obs['patient_id'] = mdata.obs['patient_id']

/tmp/ipykernel_9779/149722596.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['annot4'] = mdata.obs['annot4']
/home/jovyan/my-conda-envs/Py_env2/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)


In [11]:
adata.obs['case_control'].unique()

['POSITIVE', 'RECOVERY']
Categories (2, object): ['POSITIVE', 'RECOVERY']

In [12]:
adata = adata[adata.obs['case_control']=='POSITIVE']

In [13]:
adata.obs['case_control'].unique()

['POSITIVE']
Categories (1, object): ['POSITIVE']

In [14]:
adata.obs['centre'].unique()

['Cambridge']
Categories (1, object): ['Cambridge']

In [15]:
adata.obs['WHO_severity'].unique()

['severe', 'moderate', 'critical']
Categories (3, object): ['critical', 'moderate', 'severe']

In [16]:
adata.obs['annot4'].unique()

['pDC', 'CD14mono_anti_inflammatory', 'RBC', 'CD16mono', 'CD14mono', ..., 'Int.mono', 'CD14mono_activated', 'DC1', 'DC3_IFN', 'ASDC']
Length: 17
Categories (17, object): ['ASDC', 'CD14mono', 'CD14mono_IFN', 'CD14mono_activated', ..., 'Int.mono', 'Platelet', 'RBC', 'pDC']

In [17]:
adata.obs['days_from_steroid'].unique()

array([ -1,   7,   5,   2,  18,  14,  -5,  11, -10,   9,  -3,   1,  -6,
         6,   4,  -4,   0,  13,  -2,  -8])

In [18]:
adata.obs['annot4'] = adata.obs['annot4'].str.replace('CD14mono_activated','CD14mono')
adata.obs['annot4'] = adata.obs['annot4'].str.replace('CD14mono_anti_inflammatory','CD14mono_alt_act')

/tmp/ipykernel_9779/2951339596.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['annot4'] = adata.obs['annot4'].str.replace('CD14mono_activated','CD14mono')


In [19]:
list(adata.obs['annot4'].unique())

['pDC',
 'CD14mono_alt_act',
 'RBC',
 'CD16mono',
 'CD14mono',
 'CD14mono_IFN',
 'DC2',
 'DC3',
 'Platelet',
 'CD16mono_IFN',
 'HSPC',
 'CD16mono_C1',
 'Int.mono',
 'DC1',
 'DC3_IFN',
 'ASDC']

In [20]:
adata = adata[adata.obs['annot4'].isin(['CD14mono','CD14mono_IFN', 'CD14mono_alt_act','CD16mono','CD16mono_C1','CD16mono_IFN','Int.mono'])]

In [21]:
list(adata.obs['annot4'].unique())

['CD14mono_alt_act',
 'CD16mono',
 'CD14mono',
 'CD14mono_IFN',
 'CD16mono_IFN',
 'CD16mono_C1',
 'Int.mono']

In [22]:
adata.obs['days_from_steroid'].unique()

array([  7,  -1,   5,   2,  18,  14,  -5,  11, -10,   9,  -3,   1,  -6,
         6,   4,  -4,   0,  13,  -2,  -8])

In [23]:
adata.obs['steroid_celltype'] = 'uk'
adata.obs.loc[(adata.obs["annot4"]=='CD14mono') &(adata.obs["days_from_steroid"]<=0), "steroid_celltype"] = 'CD14mono_before'
adata.obs.loc[(adata.obs["annot4"]=='CD14mono') &(adata.obs["days_from_steroid"]>0), "steroid_celltype"] = 'CD14mono_after'

adata.obs.loc[(adata.obs["annot4"]=='CD14mono_alt_act') &(adata.obs["days_from_steroid"]<=0), "steroid_celltype"] = 'CD14mono_alt_act_before'
adata.obs.loc[(adata.obs["annot4"]=='CD14mono_alt_act') &(adata.obs["days_from_steroid"]>0), "steroid_celltype"] = 'CD14mono_alt_act_after'


adata.obs.loc[(adata.obs["annot4"]=='CD14mono_IFN') &(adata.obs["days_from_steroid"]<=0), "steroid_celltype"] = 'CD14mono_IFN_before'
adata.obs.loc[(adata.obs["annot4"]=='CD14mono_IFN') &(adata.obs["days_from_steroid"]>0), "steroid_celltype"] = 'CD14mono_IFN_after'

adata.obs.loc[(adata.obs["annot4"]=='CD16mono') &(adata.obs["days_from_steroid"]<=0), "steroid_celltype"] = 'CD16mono_before'
adata.obs.loc[(adata.obs["annot4"]=='CD16mono') &(adata.obs["days_from_steroid"]>0), "steroid_celltype"] = 'CD16mono_after'


adata.obs.loc[(adata.obs["annot4"]=='Int.mono') &(adata.obs["days_from_steroid"]<=0), "steroid_celltype"] = 'Int.mono_before'
adata.obs.loc[(adata.obs["annot4"]=='Int.mono') &(adata.obs["days_from_steroid"]>0), "steroid_celltype"] = 'Int.mono_after'

adata.obs.loc[(adata.obs["annot4"]=='CD16mono_IFN') &(adata.obs["days_from_steroid"]<=0), "steroid_celltype"] = 'CD16mono_IFN_before'
adata.obs.loc[(adata.obs["annot4"]=='CD16mono_IFN') &(adata.obs["days_from_steroid"]>0), "steroid_celltype"] = 'CD16mono_IFN_after'

adata.obs.loc[(adata.obs["annot4"]=='CD16mono_C1') &(adata.obs["days_from_steroid"]<=0), "steroid_celltype"] = 'CD16mono_C1_before'
adata.obs.loc[(adata.obs["annot4"]=='CD16mono_C1') &(adata.obs["days_from_steroid"]>0), "steroid_celltype"] = 'CD16mono_C1_after'


adata.obs['steroid_celltype'] = adata.obs['steroid_celltype'].astype('category')

/tmp/ipykernel_9779/86879094.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['steroid_celltype'] = 'uk'


In [24]:
list(adata.obs['steroid_celltype'].unique())

['CD14mono_alt_act_after',
 'CD16mono_after',
 'CD14mono_after',
 'CD14mono_IFN_before',
 'CD16mono_IFN_before',
 'CD16mono_C1_after',
 'Int.mono_after',
 'CD16mono_C1_before',
 'CD14mono_IFN_after',
 'CD16mono_IFN_after',
 'CD14mono_before',
 'Int.mono_before',
 'CD16mono_before',
 'CD14mono_alt_act_before']

In [25]:
'C116' in list(adata.obs['patient_id'].unique())

False

In [26]:
'C139' in list(adata.obs['patient_id'].unique())

False

In [27]:
'C141' in list(adata.obs['patient_id'].unique())

False

In [28]:
sc.pp.filter_cells(adata, min_genes=200)
#sc.pp.filter_genes(adata, min_cells=10)

In [29]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
adata.var['ribo'] = adata.var_names.str.startswith(("Rps","Rpl"))
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt','ribo'],percent_top=None, log1p=False, inplace=True)

In [30]:
adata= adata[adata.obs.pct_counts_mt < 10, :]
adata

View of AnnData object with n_obs × n_vars = 12802 × 33559
    obs: 'annot4', 'centre', 'case_control', 'WHO_severity', 'days_from_steroid', 'sample_id', 'patient_id', 'steroid_celltype', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo'
    var: 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [31]:
adata.X.max()

2010.3589

In [32]:
sc.pp.normalize_total(adata, target_sum=1e4)

/home/jovyan/.local/lib/python3.8/site-packages/scanpy-1.9.1-py3.8.egg/scanpy/preprocessing/_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [33]:
sc.pp.log1p(adata)

In [34]:
target_genes.head()

,GC,HDL,IFNb,IFNg,IFNg_TNF,IL10,IL13,IL4,IL4_upLPS,LA,...,TNF_P3C,TNF_PGE2,TPP,TPP_IFNb,TPP_IFNb_IFNg,sLPS,sLPS_IC,sLPS_IFNg,upLPS,upLPS_IC
0,GLDN,LOC653352,C9orf38,GBP5,CSF1,HCFC1R1,ALOX15,ALOX15,ALOX15,MMP9,...,LRG1,GBP5,RGS16,C1QTNF1,PAMR1,CXCR7,MYBPC3,GBP5,PCNX,SRPX
1,SRPX,YIPF6,ACY3,LOC400759,CCL5,SPOCD1,FOXQ1,FOXQ1,IL17RB,UCK2,...,CXCR5,PCNX,NFS1,ZSWIM4,CXCR7,PTPRF,IL2RA,PCNX,ANG,TBL2
2,ZBTB16,MOGAT1,MGAT4B,CXCL9,C20orf123,P8,PPP1R14A,PPP1R14A,CCL26,FABP5,...,ALPL,GBP4,CCL5,PCNX,PTPRF,FAM108C1,CXCR7,GCH1,RNASE4,LRG1
3,TBC1D16,PRPF40A,LOC732424,GBP1,IL2RG,ADAMTSL4,DHRS11,MAOA,WNT5A,MMP7,...,CXCR7,RCN1,CYP27B1,FAM108C1,C1QTNF1,KIAA1199,PCNX,LOC399715,SPARC,MS4A6E
4,LOC647108,LOC642033,PITX2,GBP4,GBP4,CTSG,IL1R1,SOCS1,TGM2,ACADVL,...,NFS1,SLC5A3,GPR124,CXCR7,ZSWIM4,EMR3,MLLT6,MYBPC3,FTHL12,C1S


In [35]:
len(target_genes.columns)

28

In [36]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [37]:
overlap =[]
for i in range(0, len(target_genes.columns)):
    sc.tl.score_genes(adata, list(target_genes[target_genes.columns[i]]), ctrl_size=50, gene_pool=None, n_bins=25, score_name=(target_genes.columns[i]), random_state=0, copy=False)
    print(target_genes.columns[i])
    print(list(target_genes[target_genes.columns[i]]))
    get_le = intersection(list(target_genes[target_genes.columns[i]]),list(adata.var.index))
    overlap.append(get_le)

GC
['GLDN', 'SRPX', 'ZBTB16', 'TBC1D16', 'LOC647108', 'DAAM2', 'TFCP2L1', 'ADAMTS2', 'CFH', 'SESN1', 'PDGFD', 'METTL7A', 'MFGE8', 'PDGFC', 'AGRP', 'PICK1', 'CPM', 'SAP30', 'PDK4', 'EPDR1', 'FCGR3A', 'XLKD1', 'MS4A6E', 'ABLIM3', 'FLRT2', 'MAN1A1', 'DACT1', 'SHE', 'ALOX15B', 'PIK3IP1', 'PON2', 'TCN2', 'SERPINF1', 'EIF4A3', 'RNASE2', 'ALCAM', 'GLUL', 'C5orf62', 'LOC652541', 'IDH1', 'GIMAP1', 'EPS8', 'ECHDC3', 'MS4A4A', 'SLC40A1', 'AMY2B', 'TPST1', 'ELANE', 'CLIC2', 'CTNS', 'ADORA3', 'SHMT1', 'MPZL2', 'P2RY12', 'PLCH1', 'GPRC5C', 'GADD45B', 'FAH', 'TIMP4', 'KLF9', 'FBLN5', 'C1QA', 'ANTXR2', 'PRKCABP', 'CCL18', 'GAL3ST4', 'ANKS1A', 'C1QB', 'SNTB1', 'CLTC', 'GFRA2', 'SIGLEC10', 'SH3PXD2B', 'ACSS1', 'IL1R2', 'TLR4', 'GBGT1', 'SLED1', 'PEBP1', 'BRPF3', 'TEX2', 'BLVRB', 'ITGA9', 'DISP1', 'RNASE3', 'SOCS1', 'TSC22D3', 'OLAH', 'SLC1A3', 'SEPP1', 'KIF16B', 'C10orf128', 'LOC653631', 'AMY2A', 'BCL2L11', 'CD164', 'TXNIP', 'TFPI', 'F13A1', 'CD7']


/home/jovyan/.local/lib/python3.8/site-packages/scanpy-1.9.1-py3.8.egg/scanpy/tools/_score_genes.py:151: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  for cut in np.unique(obs_cut.loc[gene_list]):


HDL
['LOC653352', 'YIPF6', 'MOGAT1', 'PRPF40A', 'LOC642033', 'UHMK1', 'MADCAM1', 'ASXL2', 'GOLPH4', 'LOC388458', 'HMGCR', 'ARID4B', 'FBXW7', 'CCDC110', 'GAS2L3', 'LOC727948', 'ITPRIPL2', 'AAK1', 'UGP2', 'PLEKHB2', 'TAZ', 'LOC730995', 'TSPAN12', 'RSPRY1', 'FLJ20309', 'LOC653515', 'RABGAP1', 'LOC646863', 'PAK2', 'RAD21', 'LOC649037', 'LOC196752', 'C4B', 'FGFR1', 'ABCD4', 'LOC644162', 'VPS13C', 'CDC2L2', 'PLEKHA1', 'MRAP', 'ATXN3', 'LOC649495', 'UBN2', 'FLJ44674', 'FCN2', 'HRB', 'PUM1', 'LOC441662', 'WHAMM', 'FLJ22184', 'LOC643953', 'LOC652455', 'OPA3', 'UGCGL1', 'LOC642335', 'ZBTB34', 'LOC652726', 'LOC441155', 'SPINLW1', 'RAB40C', 'LOC652802', 'NBPF20', 'BLZF1', 'SSX4', 'BAZ1A', 'NDRG4', 'LOC642628', 'ARL8A', 'INS-IGF2', 'LOC650950', 'SC4MOL', 'PHKA2', 'MLANA', 'C20orf94', 'MDM2', 'C4orf31', 'HYDIN', 'SLC12A1', 'PPTC7', 'SRFBP1', 'PDXDC2', 'TAF13', 'GNE', 'FAM181A', 'LOC649260', 'MLL5', 'ALKBH8', 'RGNEF', 'LOC643913', 'LOC730196', 'CMYA3', 'CNBD1', 'LOC651167', 'PPFIBP1', 'NUDT4P1', 'LOC

/home/jovyan/.local/lib/python3.8/site-packages/scanpy-1.9.1-py3.8.egg/scanpy/tools/_score_genes.py:151: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  for cut in np.unique(obs_cut.loc[gene_list]):


IFNb
['C9orf38', 'ACY3', 'MGAT4B', 'LOC732424', 'PITX2', 'CNDP1', 'NRG2', 'HCRTR2', 'C6orf52', 'GAGE1', 'C6orf134', 'LOC728081', 'MOG', 'TNXB', 'RKHD1', 'LOC653206', 'C16orf90', 'ZNF30', 'IFI27', 'MTTP', 'UTP20', 'LYZL4', 'LOC440013', 'MBLAC1', 'LOC644976', 'DKFZp761O2018', 'ZNF77', 'MARVELD2', 'FAM155A', 'FAM110C', 'TTC7A', 'LOC647681', 'TLR3', 'LOC645334', 'LOC650881', 'LOC643747', 'C9orf29', 'MYF5', 'LOC132707', 'LOC644975', 'LOC392221', 'AXL', 'LOC647680', 'LOC441847', 'CPXM2', 'TMC3', 'LOC128322', 'C9orf135', 'GRP', 'LOC644662', 'SRPX', 'PRAMEF1', 'PADI4', 'ZNF667', 'SNORA33', 'NLRP11', 'LOC442517', 'ZNF468', 'PHLPP2', 'PCDHB3', 'GAGE5', 'ST8SIA6', 'LOC642685', 'FLJ39061', 'NKD1', 'CCDC80', 'ATF7IP2', 'LOC644444', 'NWD1', 'CAPZA3', 'C8orf22', 'LOC283278', 'ZC3H18', 'LOC653441', 'LOC731884', 'NAV3', 'SLC35C1', 'FOXA2', 'FIBP', 'LOC648735', 'LOC643692', 'TMEM110', 'LOC642595', 'LOC732226', 'LOC647666', 'WDR73', 'NXF2B', 'KIAA1143', 'LOC440292', 'TTTY5', 'OAS2', 'HSFY1', 'DYX1C1', 'T

...

/home/jovyan/.local/lib/python3.8/site-packages/scanpy-1.9.1-py3.8.egg/scanpy/tools/_score_genes.py:151: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  for cut in np.unique(obs_cut.loc[gene_list]):


upLPS
['PCNX', 'ANG', 'RNASE4', 'SPARC', 'FTHL12', 'CRISPLD2', 'FTHL3', 'TSPAN3', 'SLC11A1', 'AQP9', 'C3orf19', 'CXCL5', 'TRAF3IP2', 'WSB1', 'CLDN23', 'TUBG2', 'NINJ1', 'SLC7A11', 'LOC283050', 'ELF4', 'BHLHB3', 'FTHL11', 'ELL2', 'SLC16A10', 'DNAJB2', 'GPSM1', 'INPP1', 'SELM', 'ACVR1B', 'CLDN15', 'LAT', 'SEPW1', 'SLAMF9', 'TMEM49', 'MLLT6', 'PHF13', 'SLC5A3', 'SLC3A2', 'FTHL8', 'HLA-E', 'ADAMDEC1', 'CFLAR', 'TREM1', 'CLDND2', 'MRPS6', 'SP140', 'ARRDC2', 'SEMA6B', 'NOTCH3', 'WASL', 'ZSWIM4', 'CARKD', 'LPP', 'NKG7', 'TMEM38B', 'FNBP4', 'LUC7L', 'TCEA2', 'MKLN1', 'KIAA0174', 'PATL1', 'FTH1', 'SRGN', 'C21orf7', 'P4HTM', 'AVPI1', 'CXCL16', 'ADARB1', 'WDFY1', 'APBB3', 'PARP14', 'STX4', 'ZFYVE16', 'MS4A14', 'LOC388681', 'ZDHHC19', 'SAFB', 'ZBTB34', 'ZNF274', 'CYB5A', 'AOX1', 'LOC727762', 'NRIP3', 'RABGEF1', 'TMEM140', 'SESTD1', 'MYBPC3', 'CCL23', 'LOC285074', 'KYNU', 'DCP1A', 'BTG1', 'ENO2', 'VNN2', 'ATG16L2', 'MTMR3', 'LOC728006', 'MITF', 'BBS1', 'GADD45G']


/home/jovyan/.local/lib/python3.8/site-packages/scanpy-1.9.1-py3.8.egg/scanpy/tools/_score_genes.py:151: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  for cut in np.unique(obs_cut.loc[gene_list]):


upLPS_IC
['SRPX', 'TBL2', 'LRG1', 'MS4A6E', 'C1S', 'DAAM2', 'TGM2', 'LOC647108', 'GDF15', 'BOK', 'TBC1D16', 'DHRS3', 'NRIP1', 'SEMA6B', 'IL2RA', 'TTLL4', 'SLC5A3', 'SYS1', 'CYB5A', 'ANKRD33', 'SOAT1', 'PLIN2', 'ENOSF1', 'GLUL', 'TPST1', 'ARHGAP12', 'CD38', 'KITLG', 'MRPS6', 'TIMP1', 'PDK4', 'CHST15', 'FAM187B', 'MLXIPL', 'MAN1A1', 'FLNB', 'ELL2', 'KCTD3', 'HTRA4', 'IDH1', 'CTSL1', 'SIGLEC11', 'EIF1B', 'FAM20C', 'FAH', 'ANKRD29', 'C1QA', 'AGAP3', 'C5orf62', 'SNTB1', 'SAT1', 'DVL3', 'FAIM3', 'PLXNA1', 'PTGER2', 'C1RL', 'CNIH4', 'THBS1', 'RRAGD', 'HSDL2', 'GUSB', 'PVALB', 'GUSBL1', 'FLJ20273', 'TCN2', 'FLJ20489', 'ADD3', 'MS4A4A', 'CTSD', 'AKR1B1', 'HIPK2', 'PALLD', 'PAPSS2', 'SLA', 'BCL7B', 'CTNS', 'ZNF330', 'CRYGD', 'APLP2', 'CYP27A1', 'CPD', 'ACSL1', 'PHCA', 'CEACAM1', 'SEPW1', 'MMD', 'TEX2', 'PPAP2B', 'DPP4', 'LAMP1', 'RHOBTB3', 'IGF2BP3', 'LRPAP1', 'HAMP', 'ITGAE', 'CRK', 'HIST3H2A', 'MESDC1', 'FAM20A', 'ETS2']


/home/jovyan/.local/lib/python3.8/site-packages/scanpy-1.9.1-py3.8.egg/scanpy/tools/_score_genes.py:151: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  for cut in np.unique(obs_cut.loc[gene_list]):


In [38]:
import csv
with open('leading_edge_genes_before_after.csv', 'w', newline='') as output:
    writer = csv.writer(output)
    writer.writerows(overlap)
 

In [ ]:
list(adata.obs['steroid_celltype'].unique())

In [ ]:
adata.obs['steroid_celltype'].cat.reorder_categories(['CD14mono_before','CD14mono_after','CD14mono_IFN_before',
                                                      'CD14mono_IFN_after', 'CD14mono_alt_act_before','CD14mono_alt_act_after',
                                                      'CD16mono_before','CD16mono_after','CD16mono_IFN_before','CD16mono_IFN_after','CD16mono_C1_before','CD16mono_C1_after',
                                                     'Int.mono_before','Int.mono_after'], inplace=True)

In [ ]:
scores = adata.obs[[s for s in list(target_genes.columns)]]
scores['group'] = adata.obs['steroid_celltype']
scores.groupby('group').median()

In [ ]:
adata.obs['steroid_celltype'].unique()

In [ ]:
import matplotlib.pyplot as plt
sns.set(font_scale=0.8) # font size 2
fig, ax = plt.subplots(figsize=(8,12))  
pal = sns.diverging_palette(150, 275, s=80, l=55, n=9)
score_mean = scores.groupby('group').mean()
g = sns.heatmap(score_mean.T, cmap=pal,  annot=False,linewidths=1,  square=True, cbar_kws={"shrink": 0.4},annot_kws={"size": 10})
#plt.ylabel("WHO_temp_severity")
plt.xlabel("timeline")
#plt.savefig('./CD16mono_final.png',dpi=300,bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt
sns.set(font_scale=1) # font size 2
fig, ax = plt.subplots(figsize=(8,12))  
pal = sns.diverging_palette(150, 275, s=80, l=55, n=9)
score_mean =scores.groupby('group').mean()
#Z_score the data
score_mean= (score_mean - score_mean.mean())/score_mean.std()

max_val = (max((np.array(score_mean)).max(0)))
min_val = (min((np.array(score_mean)).min(0)))
mid_val = (max((np.array(score_mean)).max(0))) - ((max((np.array(score_mean)).max(0))) - (min((np.array(score_mean)).min(0))))/2
g = sns.heatmap(score_mean.T, cmap=pal,  annot=False,vmin=min_val, vmax=max_val, linewidths=1, center=mid_val, square=True, cbar_kws={"shrink": 0.4},annot_kws={"size": 78})
#plt.ylabel("WHO_temp_severity")
plt.xlabel("")
plt.tight_layout()
#plt.savefig('/home/jovyan/scripts/renal_covid_19/sig_gene_plot/cytokine_sig_allmono_wave2_pos.pdf')

In [ ]:
new_scores =scores[[
     'upLPS',
'sLPS',
'sLPS_IFNg',
'IFNg',
'IFNg_TNF',
'TNF',
    'IL13',
'IL4',
'IL4_upLPS',
'sLPS_IC',
'upLPS_IC',
'IL10',
'GC',
'HDL',
'IFNb',
'P3C',
'P3C_PGE2',
'PGE2',
'TNF_P3C',
'TNF_PGE2',
'TPP',
'TPP_IFNb',
'TPP_IFNb_IFNg',
'LA',
'LiA',
'OA',
'PA',
'SA','group']]

In [ ]:
import matplotlib.pyplot as plt
sns.set(font_scale=1) # font size 2
fig, ax = plt.subplots(figsize=(8,12))  
pal = sns.diverging_palette(150, 275, s=80, l=55, n=9)
score_mean = new_scores.groupby('group').mean()
#Z_score the data
score_mean= (score_mean - score_mean.mean())/score_mean.std()

max_val = (max((np.array(score_mean)).max(0)))
min_val = (min((np.array(score_mean)).min(0)))
mid_val = (max((np.array(score_mean)).max(0))) - ((max((np.array(score_mean)).max(0))) - (min((np.array(score_mean)).min(0))))/2
g = sns.heatmap(score_mean.T, cmap=pal,  annot=False,vmin=min_val, vmax=max_val, linewidths=1, center=mid_val, square=True, cbar_kws={"shrink": 0.4},annot_kws={"size": 78})
#plt.ylabel("WHO_temp_severity")
plt.xlabel("")
plt.tight_layout()
plt.savefig('/home/jovyan/scripts/renal_covid_19/sig_gene_plot/cytokine_sig_allmono_wave2_pos.pdf')

In [ ]:
to_plot = score_mean.T

In [ ]:
to_plot.head()

In [ ]:
to_plot.to_csv('/home/jovyan/scripts/renal_covid_19/sig_gene_plot/gene_score_to_plot_allmono_wave2_pos_steroidonly.csv')

# do stats

In [ ]:
adata

In [ ]:
scores_stat = adata.obs[[s for s in list(target_genes.columns)]]
scores_stat['group'] = adata.obs['steroid_celltype']
scores_stat['sample_id'] = adata.obs['sample_id']

In [ ]:
scores_stat

In [ ]:
subset_before = scores_stat[scores_stat['group']=='Int.mono_before'].copy()
subset_after = scores_stat[scores_stat['group']=='Int.mono_after'].copy()


In [ ]:
subset_before.shape

In [ ]:
subset_after.shape

In [ ]:
list(subset_before['sample_id'].unique())

In [ ]:
subset_before.head()

In [ ]:
subset_after.head()

In [ ]:
before=subset_before.groupby('sample_id').mean().dropna()


In [ ]:
before

In [ ]:
after=subset_after.groupby('sample_id').mean().dropna()


In [ ]:
after

In [ ]:
pval=[]
from scipy.stats import ranksums
for i in range (0, len((before.columns))):
    print ((before.columns[i]))
    print ((after.columns[i]))
    p = ranksums(before[before.columns[i]].values,after[after.columns[i]].values) 
    pval.append(p[1])
    print(p)
    print ('----------')
print(pval)

In [ ]:
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector

stats = importr('stats')
pvalue_list=pval
p_adjust = stats.p_adjust(FloatVector(pvalue_list), method = 'fdr')
for i in range (0,len(after.columns)):
    print ((after.columns[i]))
    print(p_adjust[i])
    print('___')